In [1]:
import zipfile, io, json, os
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd

In [2]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
if 'basketball.zip' not in files and 'basketball.sqlite' not in files:
        api = KaggleApi()
        #config_dir = os.getcwd()
        #config_file = 'kaggle.json'
        #api.config = os.path.join(config_dir, config_file)
        api.authenticate()
        api.dataset_download_files('wyattowalsh/basketball') #downloads to cwd
elif 'basketball.zip' in files and 'basketball.sqlite' not in files:
    with zipfile.ZipFile('basketball.zip', 'r') as zip_ref:
        zip_ref.extractall()
    os.remove('daily_execution_pipeline.yml')
    os.remove('monthly_execution_pipeline.yml')

if 'basketball.sqlite' in files:
    %load_ext sql
    %sql sqlite:///basketball.sqlite

In [36]:
%%sql
    SELECT DISTINCT TEAM_ID_HOME
    FROM Game
    WHERE Season >= 2019

 * sqlite:///basketball.sqlite
Done.


TEAM_ID_HOME
1610612746
1610612761
1610612755
1610612759
1610612756
1610612751
1610612753
1610612748
1610612757
1610612762


In [120]:
%%sql team_stats_1610612746 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612746
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612746
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612746


In [121]:
team_stats_1610612746

Game_Num_Home,TEAM_ID,TEAM_ABBREVIATION,FGM_HOME_avg,FGA_HOME_avg,FG3M_HOME_avg,FG3M_HOME_avg:1,FTM_HOME_avg,FTA_HOME_avg,OREB_HOME_avg,DREB_HOME_avg,AST_HOME_avg,STL_HOME_avg,TOV_HOME_avg,PF_HOME_avg,PTS_HOME_avg,home_win,Game_Num_Away,FGM_AWAY_avg,FGA_AWAY_avg,FG3M_AWAY_avg,FG3M_AWAY_avg:1,FTM_AWAY_avg,FTA_AWAY_avg,OREB_AWAY_avg,DREB_AWAY_avg,AST_AWAY_avg,STL_AWAY_avg,TOV_AWAY_avg,PF_AWAY_avg,PTS_AWAY_avg,away_win
1,1610612746,LAC,None,None,None,None,None,None,None,None,None,None,None,None,None,1,1,None,None,None,None,None,None,None,None,None,None,None,None,None,1
2,1610612746,LAC,42.0,81.0,11.0,11.0,17.0,24.0,11.0,34.0,24.0,8.0,14.0,25.0,112.0,1,2,50.0,80.0,18.0,18.0,23.0,30.0,3.0,36.0,33.0,7.0,20.0,24.0,141.0,0
3,1610612746,LAC,41.0,82.0,11.5,11.5,18.0,22.5,10.0,34.5,23.5,9.5,18.0,24.5,111.5,1,3,45.0,83.5,14.5,14.5,27.0,34.5,6.5,33.5,26.5,7.5,16.0,28.0,131.5,0
4,1610612746,LAC,40.333333333333336,85.0,9.333333333333334,9.333333333333334,18.666666666666668,24.0,10.333333333333334,35.0,21.0,9.333333333333334,16.0,24.0,108.66666666666667,1,4,43.0,83.0,12.333333333333334,12.333333333333334,21.333333333333332,27.0,6.333333333333333,32.333333333333336,24.0,6.0,17.333333333333332,26.0,119.66666666666667,0
5,1610612746,LAC,39.25,86.75,8.75,8.75,20.5,26.5,12.25,35.25,19.5,8.75,14.25,24.25,107.75,0,5,41.75,86.75,11.75,11.75,17.75,23.75,7.5,34.75,22.25,5.25,16.25,24.75,113.0,0
6,1610612746,LAC,39.6,89.0,9.6,9.6,22.2,28.4,12.6,36.2,19.8,7.8,15.2,25.0,111.0,1,6,42.6,86.2,11.6,11.6,19.0,25.2,7.2,34.0,23.4,5.4,16.2,24.2,115.8,1
7,1610612746,LAC,39.5,89.0,8.666666666666666,8.666666666666666,22.666666666666668,28.666666666666668,12.666666666666666,37.5,19.833333333333332,7.5,15.5,23.166666666666668,110.33333333333333,1,7,42.333333333333336,88.0,11.5,11.5,19.333333333333332,26.0,8.833333333333334,34.833333333333336,23.166666666666668,6.5,15.833333333333334,24.666666666666668,115.5,1
8,1610612746,LAC,38.714285714285715,89.42857142857143,8.571428571428571,8.571428571428571,22.571428571428573,28.857142857142858,13.714285714285714,38.714285714285715,20.571428571428573,7.714285714285714,16.428571428571427,22.857142857142858,108.57142857142857,1,8,42.57142857142857,89.0,11.857142857142858,11.857142857142858,19.285714285714285,25.857142857142858,9.428571428571429,34.142857142857146,24.142857142857142,7.142857142857143,15.857142857142858,24.571428571428573,116.28571428571429,0
9,1610612746,LAC,40.125,89.875,9.625,9.625,23.875,29.625,13.5,39.0,21.75,8.0,17.0,23.5,113.75,1,9,41.75,89.375,11.875,11.875,18.5,24.25,9.375,34.75,24.0,7.0,15.75,23.625,113.875,0
10,1610612746,LAC,39.55555555555556,89.55555555555556,9.666666666666666,9.666666666666666,22.333333333333332,27.88888888888889,13.444444444444445,38.666666666666664,21.88888888888889,7.888888888888889,17.0,23.666666666666668,111.11111111111111,1,10,40.77777777777778,89.88888888888889,11.444444444444445,11.444444444444445,18.333333333333332,23.666666666666668,9.666666666666666,34.888888888888886,23.333333333333332,7.222222222222222,16.333333333333332,23.11111111111111,111.33333333333333,1


In [122]:
%%sql team_stats_1610612761 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612761
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612761
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612761


In [123]:
%%sql team_stats_1610612755 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg.
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612755
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612755
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
(sqlite3.OperationalError) near ".": syntax error
[SQL: WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_a

In [124]:
%%sql team_stats_1610612759 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612759
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612759
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612759


In [125]:
%%sql team_stats_1610612756 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612756
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg.
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612756
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
(sqlite3.OperationalError) near ".": syntax error
[SQL: WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_a

In [126]:
%%sql team_stats_1610612751 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612751
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612751
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612751


In [127]:
%%sql team_stats_1610612753 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612753
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612753
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612753


In [128]:
%%sql team_stats_1610612748 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612748
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612748
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612748


In [129]:
%%sql team_stats_1610612757 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612757
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612757
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612757


In [130]:
%%sql team_stats_1610612762 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612762
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612762
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612762


In [131]:
%%sql team_stats_1610612754 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612754
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612754
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612754


In [132]:
%%sql team_stats_1610612742 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612742
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612742
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612742


In [133]:
%%sql team_stats_1610612766 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612766
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612766
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612766


In [134]:
%%sql team_stats_1610612744 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612744
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612744
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612744


In [135]:
%%sql team_stats_1610612745 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612745
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612745
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612745


In [136]:
%%sql team_stats_1610612765 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612765
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612765
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612765


In [137]:
%%sql team_stats_1610612747 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612747
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612747
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612747


In [138]:
%%sql team_stats_1610612738 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612738
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612738
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612738


In [139]:
%%sql team_stats_1610612743 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612743
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612743
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612743


In [140]:
%%sql team_stats_1610612760 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612760
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612760
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612760


In [141]:
%%sql team_stats_1610612764 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612764
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612764
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612764


In [142]:
%%sql team_stats_1610612758 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612758
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612758
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612758


In [143]:
%%sql team_stats_1610612763 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612763
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612763
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612763


In [144]:
%%sql team_stats_1610612740 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612740
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612740
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612740


In [145]:
%%sql team_stats_1610612741 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612741
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612741
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612741


In [146]:
%%sql team_stats_1610612752 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612752
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612752
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612752


In [147]:
%%sql team_stats_1610612739 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612739
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612739
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612739


In [148]:
%%sql team_stats_1610612749 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612749
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612749
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612749


In [149]:
%%sql team_stats_1610612737 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612737
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612737
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612737


In [150]:
%%sql team_stats_1610612750 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612750
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_AWAY = 1610612750
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612750


In [151]:
# Convert team result sets into dataframes
team_df_1610612750 = team_stats_1610612750.DataFrame()
team_df_1610612737 = team_stats_1610612737.DataFrame()
team_df_1610612749 = team_stats_1610612749.DataFrame()
team_df_1610612739 = team_stats_1610612739.DataFrame()
team_df_1610612752 = team_stats_1610612752.DataFrame()
team_df_1610612741 = team_stats_1610612741.DataFrame()
team_df_1610612740 = team_stats_1610612740.DataFrame()
team_df_1610612763 = team_stats_1610612763.DataFrame()
team_df_1610612758 = team_stats_1610612758.DataFrame()
team_df_1610612764 = team_stats_1610612764.DataFrame()
team_df_1610612760 = team_stats_1610612760.DataFrame()
team_df_1610612743 = team_stats_1610612743.DataFrame()
team_df_1610612738 = team_stats_1610612738.DataFrame()
team_df_1610612747 = team_stats_1610612747.DataFrame()
team_df_1610612765 = team_stats_1610612765.DataFrame()
team_df_1610612745 = team_stats_1610612745.DataFrame()
team_df_1610612744 = team_stats_1610612744.DataFrame()
team_df_1610612766 = team_stats_1610612766.DataFrame()
team_df_1610612742 = team_stats_1610612742.DataFrame()
team_df_1610612754 = team_stats_1610612754.DataFrame()
team_df_1610612762 = team_stats_1610612762.DataFrame()
team_df_1610612757 = team_stats_1610612757.DataFrame()
team_df_1610612748 = team_stats_1610612748.DataFrame()
team_df_1610612753 = team_stats_1610612753.DataFrame()
team_df_1610612751 = team_stats_1610612751.DataFrame()
team_df_1610612756 = team_stats_1610612756.DataFrame()
team_df_1610612759 = team_stats_1610612759.DataFrame()
team_df_1610612755 = team_stats_1610612755.DataFrame()
team_df_1610612761 = team_stats_1610612761.DataFrame()
team_df_1610612746 = team_stats_1610612746.DataFrame()

In [152]:
# Union the game data for all the teams
comb_df = pd.concat([team_df_1610612750, team_df_1610612737, team_df_1610612749, team_df_1610612739, team_df_1610612752, 
          team_df_1610612741, team_df_1610612740, team_df_1610612763, team_df_1610612758, team_df_1610612764, 
         team_df_1610612760, team_df_1610612743, team_df_1610612738, team_df_1610612747, team_df_1610612765, 
         team_df_1610612745, team_df_1610612744, team_df_1610612766, team_df_1610612742, team_df_1610612754, 
         team_df_1610612762, team_df_1610612757, team_df_1610612748, team_df_1610612753, team_df_1610612751, 
         team_df_1610612756, team_df_1610612759, team_df_1610612755, team_df_1610612761, team_df_1610612746])
comb_df.head()

,Game_Num_Home,TEAM_ID,TEAM_ABBREVIATION,FGM_HOME_avg,FGA_HOME_avg,FG3M_HOME_avg,FG3M_HOME_avg:1,FTM_HOME_avg,FTA_HOME_avg,OREB_HOME_avg,...,FTM_AWAY_avg,FTA_AWAY_avg,OREB_AWAY_avg,DREB_AWAY_avg,AST_AWAY_avg,STL_AWAY_avg,TOV_AWAY_avg,PF_AWAY_avg,PTS_AWAY_avg,away_win
0,1,1610612750,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2,1610612750,MIN,39.000000,91.000000,13.000000,13.000000,25.000000,33.00,14.000000,...,22.0,28.0,12.000000,40.000000,23.000000,9.0,13.000000,22.000000,127.000000,1.0
2,3,1610612750,MIN,38.000000,97.000000,13.000000,13.000000,22.000000,27.50,13.500000,...,22.5,29.5,12.500000,39.500000,26.500000,10.0,15.000000,21.500000,124.000000,0.0
3,4,1610612750,MIN,41.333333,101.666667,11.666667,11.666667,21.333333,28.00,15.666667,...,22.0,29.0,10.333333,35.666667,21.666667,11.0,17.333333,21.333333,114.333333,1.0
4,5,1610612750,MIN,39.500000,100.500000,10.250000,10.250000,22.000000,29.25,15.500000,...,19.5,26.0,8.500000,37.750000,24.500000,10.5,18.500000,21.750000,118.500000,0.0
